#### Score: 34.6562

https://www.kaggle.com/poiupoiu/how-to-use-sarimax

https://habr.com/ru/post/210530/

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font='Arial')
import numpy as np
import statsmodels.api as sm

In [3]:
train = pd.read_csv('data_dengue/dengue_features_train.csv' ,parse_dates=['week_start_date'],index_col='week_start_date')
test = pd.read_csv('data_dengue/dengue_features_test.csv', parse_dates=['week_start_date'],index_col='week_start_date')
labels = pd.read_csv('data_dengue/dengue_labels_train.csv' , index_col=None)

In [4]:
ts = pd.concat([train.reset_index(),labels["total_cases"]], axis=1)

In [5]:
ts = ts.set_index("week_start_date")

In [8]:
ts_sj=ts[ts.city=='sj']
ts_iq=ts[ts.city=='iq']

In [12]:
ts_cases_sj = ts_sj["total_cases"]
ts_cases_iq = ts_iq["total_cases"]

In [25]:
ts_cases_sj.shape

(936,)

In [32]:
ts_cases_iq.shape

(520,)

In [13]:
%%time
my_order = (12, 1, 1)
my_seasonal_order = (5, 1, 5, 4)
#my_order = (12, 1, 12)
#my_seasonal_order = (5, 1, 5, 52)
ts_sj_float = ts_cases_sj.astype("float64")
ts_iq_float = ts_cases_iq.astype("float64")
model_sj = sm.tsa.statespace.SARIMAX(ts_sj_float, order=my_order, seasonal_order=my_seasonal_order).fit()
model_iq = sm.tsa.statespace.SARIMAX(ts_iq_float, order=my_order, seasonal_order=my_seasonal_order).fit()

D:\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
D:\Miniconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
D:\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
D:\Miniconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Wall time: 1min 53s


In [34]:
predict_sj = model_sj.predict(start=937, end=937+259)
predict_iq = model_iq.predict(start=521, end=521+155)

D:\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:576: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)
D:\Miniconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:576: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


In [46]:
test_df = pd.read_csv('data_dengue/dengue_features_test.csv', index_col=None)
result_columns_sj = test_df[test_df['city']=='sj'][['city','year','weekofyear']]
result_columns_iq = test_df[test_df['city']=='iq'][['city','year','weekofyear']]

In [47]:
result_columns_sj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260 entries, 0 to 259
Data columns (total 3 columns):
city          260 non-null object
year          260 non-null int64
weekofyear    260 non-null int64
dtypes: int64(2), object(1)
memory usage: 8.1+ KB


In [48]:
result_cases_sj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 937 to 1196
Data columns (total 1 columns):
total_cases    260 non-null float64
dtypes: float64(1)
memory usage: 2.2 KB


In [33]:
result_columns_iq.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 156 entries, 2010-07-02 to 2013-06-25
Data columns (total 3 columns):
city          156 non-null object
year          156 non-null int64
weekofyear    156 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.9+ KB


In [61]:
result_cases_sj = pd.DataFrame(predict_sj)
result_cases_sj.columns = ['total_cases']
result_cases_sj = result_cases_sj.reset_index()
result_df_sj = pd.concat([result_columns_sj, result_cases_sj], axis=1)
result_df_sj['total_cases'] = result_df_sj['total_cases'].round().astype(int)
result_df_sj = result_df_sj.drop(['index'], axis=1)

In [62]:
result_df_sj.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,3
1,sj,2008,19,3
2,sj,2008,20,3
3,sj,2008,21,5
4,sj,2008,22,6


In [65]:
result_cases_iq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 2 columns):
index          156 non-null int64
total_cases    156 non-null float64
dtypes: float64(1), int64(1)
memory usage: 2.6 KB


In [68]:
result_cases_iq = pd.DataFrame(predict_iq)
result_cases_iq.columns = ['total_cases']
result_cases_iq = result_cases_iq.reset_index()
result_columns_iq = result_columns_iq.reset_index()
result_df_iq = pd.concat([result_columns_iq, result_cases_iq], axis=1)
result_df_iq['total_cases'] = result_df_iq['total_cases'].round().astype(int)
result_df_iq = result_df_iq.drop(['index'], axis=1)

In [71]:
result_df_iq.head()

,city,year,weekofyear,total_cases
0,iq,2010,26,2
1,iq,2010,27,4
2,iq,2010,28,5
3,iq,2010,29,3
4,iq,2010,30,2


In [72]:
result = pd.concat([result_df_sj,result_df_iq], axis=0)

In [73]:
result.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,3
1,sj,2008,19,3
2,sj,2008,20,3
3,sj,2008,21,5
4,sj,2008,22,6


In [74]:
result.to_csv('sarima_2_city.csv', index=False)